In [1]:
from basicpy import BaSiC
from basicpy import datasets as bdata
import tifffile
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
from skimage import io
import numpy as np


In [2]:
#GET FILE LIST (works with cylinter-like structure)
paths = [
    'H://batch4_cellcycle4//cylinter//',
    'H://batch4_cellcycle4//cylinter_12//'
]

# Lists to hold the full paths of files
tif_files = []
mask_files = []

# Iterate through each path
for path in paths:
    # Construct the paths for the tif and mask folders
    tif_path = os.path.join(path, 'tif')
    mask_path = os.path.join(path, 'mask')

    # Check if the tif folder exists and collect the files
    if os.path.exists(tif_path):
        for root, dirs, files in os.walk(tif_path):
            for file in files:
                tif_files.append(os.path.join(root, file))

    # Check if the mask folder exists and collect the files
    if os.path.exists(mask_path):
        for root, dirs, files in os.walk(mask_path):
            for file in files:
                mask_files.append(os.path.join(root, file))

In [4]:
#Index the markers to be corrected
marker_list = ['DNA1', 'CyclinE', 'DNA2', 'pChk1', 'DNA3', 'Tcf1', 'pRb', 'CyclinB1', 'DNA4', 'CyclinA',
    'CD11c', 'p27', 'Vimentin', 'DNA5', 'TIM-3', 'PanCK', 'Wee1', 'DNA6', 'CD163', 'CyclinD1',
    'p21', 'DNA7', 'pH3', 'CD45', 'MPM-2', 'DNA8', 'pRPA', 'pStat1', 'CD20', 'DNA9', 'PCNA',
    'Geminin', 'gH2Ax', 'DNA10', 'CD4', 'aSMA', 'CD8a', 'DNA11', 'Iba1', 'PAX8', 'PD-1',
    'DNA12', 'Ki67', 'FOX-P3', 'CD31', 'DNA13', 'CD4', 'FAP', 'NKG2a']

markers_to_correct = ['CyclinE', 'pRb', 'CyclinB1', 'TIM-3', 'Wee1', 'CyclinD1','p21', 'pH3', 'MPM-2','pRPA', 'pStat1','PCNA',
    'Geminin', 'gH2Ax', 'PD-1', 'Ki67']
indexes = [marker_list.index(marker) for marker in markers_to_correct if marker in marker_list]
output_folder = 'D://basic_images_cellcycle'  # Specify your output folder path
os.makedirs(output_folder, exist_ok=True)

In [ ]:
import os
import pandas as pd
import tifffile
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure

# Define the path to the directory containing the CSV files
csv_folder = "Q://afarkkila_microscopy//Data//CellCycle//analysis//background_correction"

# Iterate over the markers to correct and read corresponding images
for image, mask in zip(tif_files, mask_files): 
    df_all = pd.DataFrame()      

    # Extract image base name for comparison
    image_clean_name = os.path.splitext(os.path.basename(image))[0].replace('.', '_')  # Replace dots with underscores
    parts = image_clean_name.split('_')
    if len(parts) >= 2:
        sample_name = '_'.join(parts[:2])

    # Check if there is a CSV file that contains the sample_name in its filename
    if any(sample_name in csv_file for csv_file in os.listdir(csv_folder) if csv_file.endswith(".csv")):
        print(f"Image {sample_name} has already been processed. Skipping...")
        continue

    mask_filled = tifffile.imread(mask) 
    for marker, index in zip(markers_to_correct, indexes):
        print("newround")
        print(image)
        print(marker)

        # Generate the output filename for the corrected image
        output_filename = f"{sample_name}_{marker}.tif"
        output_path = os.path.join(output_folder, output_filename)

        # Check if the corrected image file already exists
        if os.path.exists(output_path):
            # If the file exists, load the saved corrected image
            corrected_image_filled = tifffile.imread(output_path)
            print(f"{output_filename} already exists. Loading the existing image.")
        else:
            # If the file does not exist, perform the BaSiC correction and save the image
            image_filled = tifffile.imread(image, key=index)
            z = 1  
            expanded_array = np.expand_dims(image_filled, axis=0)  # Add a new axis at the beginning
            expanded_array = np.repeat(expanded_array, z, axis=0)

            # BaSiC Correction
            basic = BaSiC(get_darkfield=True, smoothness_flatfield=1)
            basic.fit(expanded_array)
            corrected_image_filled = basic.transform(expanded_array)

            # Save the corrected image
            tifffile.imwrite(output_path, corrected_image_filled)

            # Plot and save the result
            fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # Adjusted figsize for better layout
            im = axes[0].imshow(expanded_array[0], vmin=200, vmax=800)
            fig.colorbar(im, ax=axes[0])
            axes[0].set_title("Original")
            im = axes[1].imshow(corrected_image_filled[0], vmin=200, vmax=1000)
            fig.colorbar(im, ax=axes[1])
            axes[1].set_title("Corrected")
            fig.tight_layout()
            plot_filename = f"{sample_name}_{marker}_plot.png"
            plot_path = os.path.join(output_folder, plot_filename)
            plt.savefig(plot_path)
            plt.close(fig)

        # Quantify the properties
        properties = measure.regionprops_table(mask_filled, corrected_image_filled[0], properties=['label','intensity_mean'])
        properties = pd.DataFrame(properties)
        properties.rename(columns={'intensity_mean': marker, 'label': 'CellID'}, inplace=True)
        properties["imageid"] = sample_name
        
        corrected_image_filled= None
        expanded_array = None
        image_filled = None
        # Merge properties into the dataframe
        if df_all.empty:
            df_all = properties
        else:
            df_all = pd.merge(left=df_all, right=properties, on=['CellID', "imageid"], how='outer')

    # Save the dataframe to a CSV file
    output_filename = f"{sample_name}.csv"
    output_path = os.path.join(output_folder, output_filename)
    df_all.to_csv(output_path, index=False)  # Ensure index is not written to the CSV